# User Management Endpoints Testing

This notebook provides comprehensive testing for all user management endpoints in the FastAPI CRUD application.

## Endpoints Covered:
- GET `/users/` - List all users (admin only)
- POST `/users/` - Create new user (admin only)
- GET `/users/me` - Get current user info
- PATCH `/users/me` - Update current user
- PATCH `/users/me/password` - Update current user password
- DELETE `/users/me` - Delete current user account
- GET `/users/{user_id}` - Get user by ID
- PATCH `/users/{user_id}` - Update user by ID (admin only)
- DELETE `/users/{user_id}` - Delete user by ID (admin only)
- PATCH `/users/{user_id}/activate` - Activate user (admin only)
- PATCH `/users/{user_id}/deactivate` - Deactivate user (admin only)
- PATCH `/users/{user_id}/promote` - Promote to superuser (admin only)
- GET `/users/me/sessions` - Get user sessions
- DELETE `/users/me/sessions` - Invalidate all sessions
- GET `/users/me/profile` - Get user profile
- PATCH `/users/me/profile` - Update user profile

In [8]:
# Import required libraries
import requests
import json
from datetime import datetime
import time
from pprint import pprint
import uuid

# Configuration
BASE_URL = "http://localhost:8000"
API_V1_STR = "/api/v1"
AUTH_URL = f"{BASE_URL}{API_V1_STR}/auth"
USERS_URL = f"{BASE_URL}{API_V1_STR}/users"

# Test users data
admin_user = {"email": "admin@example.com", "password": "ChangeThis123!"}

test_user = {
    "email": f"test_user_{int(time.time())}@example.com",
    "password": "TestPassword123!",
    "first_name": "Test",
    "last_name": "User",
}

# Global variables for storing tokens and user data
admin_token = None
user_token = None
test_user_id = None
created_user_id = None

print(f"Testing user endpoints at: {USERS_URL}")
print(f"Test user email: {test_user['email']}")

Testing user endpoints at: http://localhost:8000/api/v1/users
Test user email: test_user_1753928803@example.com


## Setup: Authentication

First, let's authenticate as admin and create a test user.

### 🔧 Troubleshooting Admin Authentication

If admin authentication fails with "Incorrect email or password":

1. **Check if admin user exists**: The database might need initialization
2. **Verify credentials**: Admin password should match `.env` file (`FIRST_SUPERUSER_PASSWORD`)
3. **Run database initialization** if needed:
   ```python
   from src.core.database import init_db
   from src.api.deps import get_db
   session = next(get_db())
   init_db(session)
   ```

Current admin credentials from `.env`:
- Email: `admin@example.com` 
- Password: `ChangeThis123!`

In [ ]:
# Setup: Get admin token
def setup_admin_auth():
    global admin_token

    login_data = {"username": admin_user["email"], "password": admin_user["password"]}

    response = requests.post(f"{AUTH_URL}/login/access-token", data=login_data)

    if response.status_code == 200:
        data = response.json()
        admin_token = data.get("access_token")
        print(f"✅ Admin authenticated successfully")
        print(f"Admin Token: {admin_token[:50]}...")
        return True
    else:
        print(f"❌ Admin authentication failed (Status: {response.status_code})")
        print(f"Response: {response.text}")
        if response.status_code == 400:
            print("\n🔍 Troubleshooting:")
            print("- Check if admin user exists in database")
            print("- Verify password matches .env FIRST_SUPERUSER_PASSWORD")
            print("- Run database initialization if needed")
        return False


# Setup: Create and authenticate test user
def setup_test_user():
    global user_token, test_user_id

    # Create test user via signup
    signup_response = requests.post(f"{AUTH_URL}/signup", json=test_user)

    if signup_response.status_code == 201:
        user_data = signup_response.json()
        test_user_id = user_data.get("id")
        print(f"✅ Test user created: {test_user_id}")

        # Login test user
        login_data = {"username": test_user["email"], "password": test_user["password"]}

        login_response = requests.post(
            f"{AUTH_URL}/login/access-token", data=login_data
        )

        if login_response.status_code == 200:
            data = login_response.json()
            user_token = data.get("access_token")
            print(f"✅ Test user authenticated successfully")
            print(f"User Token: {user_token[:50]}...")
            return True

    print(f"❌ Test user setup failed")
    return False


# Run setup
print("Setting up authentication...")
admin_success = setup_admin_auth()
user_success = setup_test_user()

if admin_success and user_success:
    print("\n🎉 Setup completed successfully!")
else:
    print("\n❌ Setup failed. Please check server and credentials.")

Setting up authentication...
✅ Admin authenticated successfully
Admin Token: eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJleHAiOjE3N...
✅ Admin authenticated successfully
Admin Token: eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJleHAiOjE3N...
✅ Test user created: None
✅ Test user authenticated successfully
User Token: eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJleHAiOjE3N...

🎉 Setup completed successfully!
✅ Test user created: None
✅ Test user authenticated successfully
User Token: eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJleHAiOjE3N...

🎉 Setup completed successfully!


## 1. Get All Users (Admin Only)

Test listing all users with pagination and filtering.

In [10]:
# Test get all users
def test_get_all_users():
    if not admin_token:
        print("❌ Admin token not available")
        return None

    headers = {"Authorization": f"Bearer {admin_token}"}

    # Test basic listing
    print("1. Basic user listing:")
    response = requests.get(USERS_URL, headers=headers)
    print(f"Status: {response.status_code}")

    if response.status_code == 200:
        data = response.json()
        print(f"Total users: {data.get('count', 0)}")
        print(f"Users returned: {len(data.get('data', []))}")
        pprint(data)
    else:
        print(f"Error: {response.text}")

    # Test with pagination
    print("\n2. With pagination:")
    params = {"skip": 0, "limit": 2}
    response = requests.get(USERS_URL, headers=headers, params=params)
    print(f"Status: {response.status_code}")
    if response.status_code == 200:
        data = response.json()
        print(f"Users with limit 2: {len(data.get('data', []))}")

    # Test with filters
    print("\n3. With filters (active users):")
    params = {"is_active": True}
    response = requests.get(USERS_URL, headers=headers, params=params)
    print(f"Status: {response.status_code}")
    if response.status_code == 200:
        data = response.json()
        print(f"Active users: {len(data.get('data', []))}")


test_get_all_users()

1. Basic user listing:
Status: 200
Total users: 7
Users returned: 7
{'count': 7,
 'data': [{'created_at': '2025-07-31T02:01:43.648768',
           'email': 'test_auth_1753927272@example.com',
           'first_name': 'Test',
           'id': '643b38da-6e7f-4241-aaf9-16abe9529314',
           'is_active': True,
           'is_superuser': False,
           'last_name': 'User',
           'updated_at': '2025-07-31T02:01:43.648772'},
          {'created_at': '2025-07-31T02:06:16.274335',
           'email': 'test_auth_1753927565@example.com',
           'first_name': 'Test',
           'id': '3561f7b9-eaba-4e42-ab79-f0eb33ade2fc',
           'is_active': True,
           'is_superuser': False,
           'last_name': 'User',
           'updated_at': '2025-07-31T02:06:16.274339'},
          {'created_at': '2025-07-31T02:11:06.022553',
           'email': 'suite_test_1753927865@example.com',
           'first_name': 'Suite',
           'id': '25ec6a0f-f0b1-4c74-aaa7-3bdfb6230b13',
          

## 2. Create New User (Admin Only)

Test creating new users as admin.

In [11]:
# Test create new user
def test_create_user():
    global created_user_id

    if not admin_token:
        print("❌ Admin token not available")
        return None

    headers = {"Authorization": f"Bearer {admin_token}"}

    new_user = {
        "email": f"admin_created_{int(time.time())}@example.com",
        "password": "AdminCreated123!",
        "first_name": "Admin",
        "last_name": "Created",
        "is_active": True,
        "is_superuser": False,
    }

    response = requests.post(USERS_URL, headers=headers, json=new_user)

    print(f"POST {USERS_URL}")
    print(f"Status: {response.status_code}")

    if response.status_code == 201:
        print("✅ User created successfully!")
        data = response.json()
        created_user_id = data.get("id")
        print(f"Created user ID: {created_user_id}")
        pprint(data)
        return data
    else:
        print("❌ User creation failed!")
        print(f"Error: {response.text}")
        return None


create_user_result = test_create_user()

POST http://localhost:8000/api/v1/users
Status: 201
✅ User created successfully!
Created user ID: 7e2777fb-5276-4997-a3b5-ac2cf85e969a
{'created_at': '2025-07-31T02:27:51.070835',
 'email': 'admin_created_1753928870@example.com',
 'first_name': 'Admin',
 'id': '7e2777fb-5276-4997-a3b5-ac2cf85e969a',
 'is_active': True,
 'is_superuser': False,
 'last_name': 'Created',
 'updated_at': '2025-07-31T02:27:51.070839'}


## 3. Current User Operations

Test operations for the currently authenticated user.

In [12]:
# Test current user operations
def test_current_user_operations():
    if not user_token:
        print("❌ User token not available")
        return

    headers = {"Authorization": f"Bearer {user_token}"}

    # 1. Get current user info
    print("1. Get current user info:")
    response = requests.get(f"{USERS_URL}/me", headers=headers)
    print(f"Status: {response.status_code}")
    if response.status_code == 200:
        print("✅ Current user info retrieved")
        current_user_data = response.json()
        pprint(current_user_data)
    else:
        print(f"❌ Error: {response.text}")
        return

    # 2. Update current user
    print("\n2. Update current user:")
    update_data = {"first_name": "Updated", "last_name": "TestUser"}
    response = requests.patch(f"{USERS_URL}/me", headers=headers, json=update_data)
    print(f"Status: {response.status_code}")
    if response.status_code == 200:
        print("✅ User updated successfully")
        updated_data = response.json()
        print(
            f"New name: {updated_data.get('first_name')} {updated_data.get('last_name')}"
        )
    else:
        print(f"❌ Error: {response.text}")

    # 3. Update password
    print("\n3. Update current user password:")
    password_data = {
        "current_password": test_user["password"],
        "new_password": "NewTestPassword123!",
    }
    response = requests.patch(
        f"{USERS_URL}/me/password", headers=headers, json=password_data
    )
    print(f"Status: {response.status_code}")
    if response.status_code == 200:
        print("✅ Password updated successfully")
        # Update test user password for future tests
        test_user["password"] = "NewTestPassword123!"
    else:
        print(f"❌ Error: {response.text}")


test_current_user_operations()

1. Get current user info:
Status: 200
✅ Current user info retrieved
{'created_at': '2025-07-31T02:27:13.953744',
 'email': 'test_user_1753928803@example.com',
 'first_name': 'Test',
 'id': 'c4b43b05-f8b2-484d-b48b-0a047fd41858',
 'is_active': True,
 'is_superuser': False,
 'last_name': 'User',
 'updated_at': '2025-07-31T02:27:13.953747'}

2. Update current user:
Status: 200
✅ User updated successfully
New name: Updated TestUser

3. Update current user password:
Status: 200
✅ Password updated successfully
Status: 200
✅ Password updated successfully


## 4. Get User by ID

Test retrieving specific users by ID.

In [14]:
# Test get user by ID
def test_get_user_by_id():
    if not user_token or not test_user_id:
        print("❌ User token or ID not available")
        return

    headers = {"Authorization": f"Bearer {user_token}"}

    # 1. Get own user by ID
    print("1. Get own user by ID:")
    response = requests.get(f"{USERS_URL}/{test_user_id}", headers=headers)
    print(f"Status: {response.status_code}")
    if response.status_code == 200:
        print("✅ Own user retrieved successfully")
        data = response.json()
        print(
            f"User: {data.get('first_name')} {data.get('last_name')} ({data.get('email')})"
        )
    else:
        print(f"❌ Error: {response.text}")

    # 2. Try to get another user (should fail for non-admin)
    if created_user_id:
        print("\n2. Try to get another user (should fail for non-admin):")
        response = requests.get(f"{USERS_URL}/{created_user_id}", headers=headers)
        print(f"Status: {response.status_code}")
        if response.status_code == 403:
            print("✅ Correctly blocked access to other user")
        else:
            print(f"❌ Unexpected result: {response.text}")

    # 3. Admin getting user by ID
    if admin_token and created_user_id:
        print("\n3. Admin getting user by ID:")
        admin_headers = {"Authorization": f"Bearer {admin_token}"}
        response = requests.get(f"{USERS_URL}/{created_user_id}", headers=admin_headers)
        print(f"Status: {response.status_code}")
        if response.status_code == 200:
            print("✅ Admin successfully retrieved user")
            data = response.json()
            print(f"User: {data.get('first_name')} {data.get('last_name')}")
        else:
            print(f"❌ Error: {response.text}")


test_get_user_by_id()

❌ User token or ID not available


## 5. Admin User Management

Test admin-only user management operations.

In [15]:
# Test admin user management
def test_admin_user_management():
    if not admin_token or not created_user_id:
        print("❌ Admin token or created user ID not available")
        return

    headers = {"Authorization": f"Bearer {admin_token}"}

    # 1. Update user by ID
    print("1. Update user by ID:")
    update_data = {"first_name": "AdminUpdated", "is_active": True}
    response = requests.patch(
        f"{USERS_URL}/{created_user_id}", headers=headers, json=update_data
    )
    print(f"Status: {response.status_code}")
    if response.status_code == 200:
        print("✅ User updated by admin")
        data = response.json()
        print(f"Updated name: {data.get('first_name')}")
    else:
        print(f"❌ Error: {response.text}")

    # 2. Deactivate user
    print("\n2. Deactivate user:")
    response = requests.patch(
        f"{USERS_URL}/{created_user_id}/deactivate", headers=headers
    )
    print(f"Status: {response.status_code}")
    if response.status_code == 200:
        print("✅ User deactivated")
        data = response.json()
        print(f"Active status: {data.get('is_active')}")
    else:
        print(f"❌ Error: {response.text}")

    # 3. Activate user
    print("\n3. Activate user:")
    response = requests.patch(
        f"{USERS_URL}/{created_user_id}/activate", headers=headers
    )
    print(f"Status: {response.status_code}")
    if response.status_code == 200:
        print("✅ User activated")
        data = response.json()
        print(f"Active status: {data.get('is_active')}")
    else:
        print(f"❌ Error: {response.text}")

    # 4. Promote to superuser
    print("\n4. Promote to superuser:")
    response = requests.patch(f"{USERS_URL}/{created_user_id}/promote", headers=headers)
    print(f"Status: {response.status_code}")
    if response.status_code == 200:
        print("✅ User promoted to superuser")
        data = response.json()
        print(f"Superuser status: {data.get('is_superuser')}")
    else:
        print(f"❌ Error: {response.text}")


test_admin_user_management()

1. Update user by ID:
Status: 200
✅ User updated by admin
Updated name: AdminUpdated

2. Deactivate user:
Status: 200
✅ User deactivated
Active status: False

3. Activate user:
Status: 200
✅ User activated
Active status: True

4. Promote to superuser:
Status: 200
✅ User promoted to superuser
Superuser status: True


## 6. Session Management

Test user session management endpoints.

In [16]:
# Test session management
def test_session_management():
    if not user_token:
        print("❌ User token not available")
        return

    headers = {"Authorization": f"Bearer {user_token}"}

    # 1. Get current user sessions
    print("1. Get current user sessions:")
    response = requests.get(f"{USERS_URL}/me/sessions", headers=headers)
    print(f"Status: {response.status_code}")
    if response.status_code == 200:
        print("✅ Sessions retrieved")
        sessions = response.json()
        print(f"Active sessions: {len(sessions)}")
        for i, session in enumerate(sessions):
            print(f"  Session {i+1}: {session.get('id', 'N/A')}")
    else:
        print(f"❌ Error: {response.text}")

    # Note: We won't test session invalidation as it would invalidate our current session
    print("\n2. Session invalidation test skipped (would invalidate current session)")
    print("   To test: requests.delete(f'{USERS_URL}/me/sessions', headers=headers)")


test_session_management()

1. Get current user sessions:
Status: 200
✅ Sessions retrieved
Active sessions: 0

2. Session invalidation test skipped (would invalidate current session)
   To test: requests.delete(f'{USERS_URL}/me/sessions', headers=headers)


## 7. Profile Management

Test user profile management endpoints.

In [17]:
# Test profile management
def test_profile_management():
    if not user_token:
        print("❌ User token not available")
        return

    headers = {"Authorization": f"Bearer {user_token}"}

    # 1. Get current user profile
    print("1. Get current user profile:")
    response = requests.get(f"{USERS_URL}/me/profile", headers=headers)
    print(f"Status: {response.status_code}")
    if response.status_code == 200:
        print("✅ Profile retrieved")
        profile = response.json()
        pprint(profile)
    elif response.status_code == 404:
        print("ℹ️ No profile found (expected for new users)")
    else:
        print(f"❌ Error: {response.text}")

    # 2. Update user profile
    print("\n2. Update user profile:")
    profile_data = {
        "bio": "This is my updated bio",
        "location": "Test City",
        "website": "https://example.com",
    }
    response = requests.patch(
        f"{USERS_URL}/me/profile", headers=headers, json=profile_data
    )
    print(f"Status: {response.status_code}")
    if response.status_code == 200:
        print("✅ Profile updated")
        updated_profile = response.json()
        print(f"Bio: {updated_profile.get('bio')}")
        print(f"Location: {updated_profile.get('location')}")
    else:
        print(f"❌ Error: {response.text}")


test_profile_management()

1. Get current user profile:
Status: 404
ℹ️ No profile found (expected for new users)

2. Update user profile:
Status: 500
❌ Error: {"detail":"Failed to update profile: 1 validation error for UserProfileOutput\n  Input should be a valid dictionary or instance of UserProfileOutput [type=model_type, input_value=<UserProfile(user_id='c4b...84d-b48b-0a047fd41858')>, input_type=UserProfile]\n    For further information visit https://errors.pydantic.dev/2.9/v/model_type"}


## 8. Error Cases and Edge Cases

Test various error scenarios and edge cases.

In [18]:
# Test error cases
def test_error_cases():
    print("Testing error cases and edge cases...\n")

    # 1. Access admin endpoint without admin privileges
    print("1. Non-admin trying to list all users:")
    if user_token:
        headers = {"Authorization": f"Bearer {user_token}"}
        response = requests.get(USERS_URL, headers=headers)
        print(f"Status: {response.status_code} (expected: 403)")
        if response.status_code == 403:
            print("✅ Correctly blocked non-admin access")

    # 2. Access endpoint without authentication
    print("\n2. Access protected endpoint without token:")
    response = requests.get(f"{USERS_URL}/me")
    print(f"Status: {response.status_code} (expected: 401)")
    if response.status_code == 401:
        print("✅ Correctly requires authentication")

    # 3. Access non-existent user
    print("\n3. Get non-existent user:")
    if admin_token:
        headers = {"Authorization": f"Bearer {admin_token}"}
        fake_uuid = str(uuid.uuid4())
        response = requests.get(f"{USERS_URL}/{fake_uuid}", headers=headers)
        print(f"Status: {response.status_code} (expected: 404)")
        if response.status_code == 404:
            print("✅ Correctly returns 404 for non-existent user")

    # 4. Create user with invalid data
    print("\n4. Create user with invalid email:")
    if admin_token:
        headers = {"Authorization": f"Bearer {admin_token}"}
        invalid_user = {"email": "invalid-email", "password": "short", "first_name": ""}
        response = requests.post(USERS_URL, headers=headers, json=invalid_user)
        print(f"Status: {response.status_code} (expected: 422)")
        if response.status_code == 422:
            print("✅ Correctly validates input data")

    # 5. Update with conflicting email
    print("\n5. Update user with existing email:")
    if user_token and admin_token:
        headers = {"Authorization": f"Bearer {user_token}"}
        conflicting_data = {"email": admin_user["email"]}
        response = requests.patch(
            f"{USERS_URL}/me", headers=headers, json=conflicting_data
        )
        print(f"Status: {response.status_code} (expected: 409)")
        if response.status_code == 409:
            print("✅ Correctly prevents email conflicts")


test_error_cases()

Testing error cases and edge cases...

1. Non-admin trying to list all users:
Status: 403 (expected: 403)
✅ Correctly blocked non-admin access

2. Access protected endpoint without token:
Status: 401 (expected: 401)
✅ Correctly requires authentication

3. Get non-existent user:
Status: 404 (expected: 404)
✅ Correctly returns 404 for non-existent user

4. Create user with invalid email:
Status: 422 (expected: 422)
✅ Correctly validates input data

5. Update user with existing email:
Status: 500 (expected: 409)


## 9. Cleanup Operations

Clean up test data (optional - for testing delete operations).

In [23]:
# Cleanup test data
def test_cleanup():
    print("Cleanup operations (testing delete functionality)...\n")

    # 1. Admin deleting user
    if admin_token and created_user_id:
        print("1. Admin deleting created user:")
        headers = {"Authorization": f"Bearer {admin_token}"}
        response = requests.delete(f"{USERS_URL}/{created_user_id}", headers=headers)
        print(f"Status: {response.status_code}")
        if response.status_code == 200:
            print("✅ User deleted successfully")
            data = response.json()
            print(f"Message: {data.get('message')}")
        else:
            print(f"❌ Error: {response.text}")

    # Note: We won't delete the test user as it would break subsequent tests
    print("\n2. Test user deletion skipped (to preserve test continuity)")
    print("   To test: requests.delete(f'{USERS_URL}/me', headers=user_headers)")


# Uncomment the line below to run cleanup
test_cleanup()

Cleanup operations (testing delete functionality)...

1. Admin deleting created user:
Status: 200
✅ User deleted successfully
Message: User deleted successfully

2. Test user deletion skipped (to preserve test continuity)
   To test: requests.delete(f'{USERS_URL}/me', headers=user_headers)


## 10. Complete Test Summary

Run a comprehensive test suite and provide detailed results.

In [24]:
# Complete test suite for user endpoints
def run_complete_user_test_suite():
    print("🧪 Running Complete User Management Test Suite")
    print("=" * 60)
    
    results = {
        "timestamp": datetime.now().isoformat(),
        "tests": {},
        "summary": {}
    }
    
    # Test authentication setup
    print("\n🔐 Authentication Setup")
    admin_auth = setup_admin_auth()
    user_auth = setup_test_user()
    
    if not (admin_auth and user_auth):
        print("❌ Authentication setup failed. Cannot proceed with tests.")
        return results
    
    # Test cases
    test_cases = [
        ("get_all_users", lambda: requests.get(USERS_URL, headers={"Authorization": f"Bearer {admin_token}"})),
        ("create_user", lambda: requests.post(USERS_URL, headers={"Authorization": f"Bearer {admin_token}"}, json={
            "email": f"suite_test_{int(time.time())}@example.com",
            "password": "SuiteTest123!",
            "first_name": "Suite",
            "last_name": "Test"
        })),
        ("get_current_user", lambda: requests.get(f"{USERS_URL}/me", headers={"Authorization": f"Bearer {user_token}"})),
        ("update_current_user", lambda: requests.patch(f"{USERS_URL}/me", headers={"Authorization": f"Bearer {user_token}"}, json={
            "first_name": "Updated"
        })),
        ("get_user_by_id", lambda: requests.get(f"{USERS_URL}/{test_user_id}", headers={"Authorization": f"Bearer {user_token}"})),
        ("get_user_sessions", lambda: requests.get(f"{USERS_URL}/me/sessions", headers={"Authorization": f"Bearer {user_token}"})),
    ]
    
    print("\n🧪 Running Test Cases")
    for test_name, test_func in test_cases:
        print(f"\n  Testing {test_name}...")
        try:
            response = test_func()
            success = 200 <= response.status_code < 300
            results["tests"][test_name] = {
                "status_code": response.status_code,
                "success": success,
                "response_size": len(response.text)
            }
            status = "✅ PASS" if success else "❌ FAIL"
            print(f"    Result: {status} (Status: {response.status_code})")
        except Exception as e:
            results["tests"][test_name] = {
                "status_code": None,
                "success": False,
                "error": str(e)
            }
            print(f"    Result: ❌ ERROR - {str(e)}")
    
    # Calculate summary
    total_tests = len(results["tests"])
    passed_tests = sum(1 for test in results["tests"].values() if test["success"])
    failed_tests = total_tests - passed_tests
    
    results["summary"] = {
        "total": total_tests,
        "passed": passed_tests,
        "failed": failed_tests,
        "success_rate": (passed_tests / total_tests * 100) if total_tests > 0 else 0
    }
    
    # Print summary
    print("\n" + "=" * 60)
    print("📊 TEST SUMMARY")
    print("=" * 60)
    print(f"Total Tests: {total_tests}")
    print(f"Passed: {passed_tests}")
    print(f"Failed: {failed_tests}")
    print(f"Success Rate: {results['summary']['success_rate']:.1f}%")
    
    if failed_tests > 0:
        print("\n❌ Failed Tests:")
        for test_name, test_result in results["tests"].items():
            if not test_result["success"]:
                error_msg = test_result.get('error') if 'error' in test_result else f"Status {test_result.get('status_code')}"
                print(f"  - {test_name}: {error_msg}")
    
    return results

# Run the complete test suite
test_suite_results = run_complete_user_test_suite()

🧪 Running Complete User Management Test Suite

🔐 Authentication Setup
✅ Admin authenticated successfully
Admin Token: eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJleHAiOjE3N...
❌ Test user setup failed
❌ Authentication setup failed. Cannot proceed with tests.
